In [1]:
#include<iostream>
#include <fstream>
#include <string>
#include <vector>
#include <iomanip>

In [2]:
#define TYPE double

# N-body simulation

In this exercise, we will implement the N-Body simulating using a direct sum to compute the forces and the Euler Method to update the positons

## Datastrucutre

* Write a generic struct for a vector
* Add a function to compute the vector's norm
* Add a constructor which initializes the vector to zero
* Overload the operators +,-, and == for a second vector and the operator * for multplication with a scalar

In [3]:
struct vector
{
    
    double x,y,z;
    
        bool operator==(const vector& rhs) const {
        return x == rhs.x
            && y == rhs.y
            && z == rhs.z;
    }

    friend std::istream& operator>>(std::istream&, vector&);
    friend std::ostream& operator<<(std::ostream&, vector&);
};

In [4]:
extern "C++" 
std::istream& operator>>(std::istream& in, vector& v) {
    return in >> v.x >> v.y >> v.z;
}


In [5]:
extern "C++"
std::ostream& operator<<(std::ostream& out, vector& v) {
    auto precision = out.precision();
    auto width = out.width();
    out << std::setw(width) << std::setprecision(precision) << v.x << "  ";
    out << std::setw(width) << std::setprecision(precision) << v.y << "  ";
    out << std::setw(width) << std::setprecision(precision) << v.z;
    return out;
}

## Class for the $N$-body simulation

* Write a function to compute the force $\mathbf{F}_i=\sum\limits_{i=0,i\neq j}^n \mathbf{F}_{ij}= \sum\limits_{i=0,i\neq j}^n g_c m_j \frac{\mathbf{r}_j-\mathbf{r}_i}{\vert \mathbf{r}_j - \mathbf{r}_i\vert^3}$ for each body using a direct sum
* Write a function to compute the velocity $\mathbf{v}_i(t_k)=\mathbf{v}_i(t_{k-1})+\Delta t \frac{\mathbf{F}_i}{m_i}$ for each body
* Write a funktion to update the new positions $\mathbf{r}_i(t_{k+1})=\mathbf{r}_i(t_k)+\mathbf{v}_i(t_k)\Delta t$ of a each body

In [7]:
class Nbody
{

private:

double gc;
size_t n;
TYPE timeStepSize;
std::vector<double> masses;
std::vector<vector> positions;
std::vector<vector> velocities;
std::vector<vector> velocitiesOld;
std::vector<vector> forces;

public:

size_t timeSteps;

//Method to compute the acceleration
void computeForces(){}

//Method to compute the velocity
void computeVelocities(){
    
     std::copy(velocities.begin(),velocities.end(),velocitiesOld.begin());
}

//Method to update the positions
void updatePositions(){}

//Method to detect collisions between bodys
void detectCollisions(){

  for (int i = 0; i < n; ++i) {
            for (int j = i + 1; j < n; ++j) {
                if (positions[i] == positions[j]) {
                    std::swap(velocities[i], velocities[j]);
                }
            }
        }

}

public:

//Constructor
Nbody(std::string& fileName) { 

std::ifstream ifs(fileName);
        if (!ifs.is_open()) {
            throw std::runtime_error("Could not open " + fileName + "!");
        }
        

//Read the gravity, the number of bodies, and the number of timesteps
ifs >> gc >> n >> timeSteps >> timeStepSize;

masses.resize(n);
positions.resize(n);
std::fill(positions.begin(), positions.end(), vector);
velocities.resize(n);
std::fill(velocities.begin(), velocities.end(), vector);
accelerations.resize(n);
std::fill(accelerations.begin(), accelerations.end(), vector);

for (int i = 0; i < n; ++i) {
            ifs >> masses[i] >> positions[i] >> velocities[i];
        }
 
        std::cout << "Contents of " << fileName << std::endl;
        std::cout << gc << ' ' << n << ' ' << timeSteps << std::endl;
        for (int i = 0; i < n; ++i) {
            std::cout << masses[i] << std::endl;
            std::cout << positions[i] << std::endl;
            std::cout << velocities[i] << std::endl;
        }
        std::cout << std::endl << "Data   :      x          y          z    |     vx         vy         vz" << std::endl;

}


friend std::ostream& operator<<(std::ostream&, Nbody&);

};

input_line_9:49:15: error: no matching constructor for initialization of 'std::ifstream' (aka 'basic_ifstream<char>')
std::ifstream ifs(fileName);
              ^   ~~~~~~~~
/usr/include/c++/6/fstream:495:7: note: candidate constructor not viable: no known conversion from 'std::string' (aka 'basic_string<char>') to 'const char *' for 1st argument
      basic_ifstream(const char* __s, ios_base::openmode __mode = ios_base::in)
      ^
/usr/include/c++/6/fstream:519:7: note: candidate constructor not viable: no known conversion from 'std::string' (aka 'basic_string<char>') to 'const std::basic_ifstream<char>' for 1st argument
      basic_ifstream(const basic_ifstream&) = delete;
      ^
/usr/include/c++/6/fstream:521:7: note: candidate constructor not viable: no known conversion from 'std::string' (aka 'basic_string<char>') to 'std::basic_ifstream<char>' for 1st argument
      basic_ifstream(basic_ifstream&& __rhs)
      ^
/usr/include/c++/6/fstream:481:7: note: candidate constructor not 

/usr/include/c++/6/iomanip:178:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching '_Setfill<char>' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __os, _Setfill<_CharT> __f)
    ^
/usr/include/c++/6/iomanip:208:5: note: candidate function not viable: no overload of 'endl' matching 'std::_Setprecision' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __os, _Setprecision __f)
    ^
/usr/include/c++/6/iomanip:238:5: note: candidate function not viable: no overload of 'endl' matching 'std::_Setw' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __os, _Setw __f)
    ^
/usr/include/c++/6/iomanip:363:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching '_Put_time<char>' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __os, _Put_time<_CharT> __f)
    ^
input_line_7:2:15: note: candidate

    operator<<(basic_ostream<char, _Traits>& __out, unsigned char __c)
    ^
/usr/include/c++/6/ostream:539:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __out, const _CharT* __s)
    ^
/usr/include/c++/6/ostream:556:5: note: candidate function not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const char* __s)
    ^
/usr/include/c++/6/ostream:569:5: note: candidate function not viable: no overload of 'endl' matching 'const signed char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const signed char* __s)
    ^
/usr/include/c++/6/ostream:574:5: note: candidate function not viable: no overload of 'endl' matching 'const unsigned char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const unsigned char* __s)
  

/usr/include/c++/6/ostream:539:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __out, const _CharT* __s)
    ^
/usr/include/c++/6/ostream:556:5: note: candidate function not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const char* __s)
    ^
/usr/include/c++/6/ostream:569:5: note: candidate function not viable: no overload of 'endl' matching 'const signed char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const signed char* __s)
    ^
/usr/include/c++/6/ostream:574:5: note: candidate function not viable: no overload of 'endl' matching 'const unsigned char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const unsigned char* __s)
    ^
/usr/include/c++/6/bits/ostream.tcc:321:5: note: candidate function not v

/usr/include/c++/6/ostream:539:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __out, const _CharT* __s)
    ^
/usr/include/c++/6/ostream:556:5: note: candidate function not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const char* __s)
    ^
/usr/include/c++/6/ostream:569:5: note: candidate function not viable: no overload of 'endl' matching 'const signed char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const signed char* __s)
    ^
/usr/include/c++/6/ostream:574:5: note: candidate function not viable: no overload of 'endl' matching 'const unsigned char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const unsigned char* __s)
    ^
/usr/include/c++/6/bits/ostream.tcc:321:5: note: candidate function not v

/usr/include/c++/6/ostream:539:5: note: candidate function [with _CharT = char, _Traits = std::char_traits<char>] not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<_CharT, _Traits>& __out, const _CharT* __s)
    ^
/usr/include/c++/6/ostream:556:5: note: candidate function not viable: no overload of 'endl' matching 'const char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const char* __s)
    ^
/usr/include/c++/6/ostream:569:5: note: candidate function not viable: no overload of 'endl' matching 'const signed char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const signed char* __s)
    ^
/usr/include/c++/6/ostream:574:5: note: candidate function not viable: no overload of 'endl' matching 'const unsigned char *' for 2nd argument
    operator<<(basic_ostream<char, _Traits>& __out, const unsigned char* __s)
    ^
/usr/include/c++/6/bits/ostream.tcc:321:5: note: candidate function not v

ename: evalue

In [ ]:
extern "C++"
std::ostream& operator<<(std::ostream& out, Nbody& nb) {
    for (int i = 0; i < nb.n; ++i) {
        out << "Body " << i + 1 << " : ";
        out << std::setprecision(6) << std::setw(9) << nb.positions[i];
        out << " | ";
        out << std::setprecision(6) << std::setw(9) << nb.velocities[i];
        out << std::endl;
    }
    return out;
}

In [ ]:
std::string fileName = "nbody.txt";


In [ ]:
Nbody nb(fileName);


In [ ]:
for(size_t t = 0 ; t < nb.timeSteps;t++){

std::cout << std::endl << "Cycle " << t << std::endl;
nb.computeForces();
nb.computeVelocities();
nb.updatePositions();
nb.detectCollision();
std::cout << nb;

}